In [1]:
import os
if os.getcwd() == '/home/user/code':
    os.chdir('/home/user/code/nlp2024_ClefTask4SOTA')

%load_ext autoreload
%autoreload 2

import transformers
transformers.set_seed(17)


2024-06-11 09:12:51.642083: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# What should be the max length of the model?

Approach: Tokenize the longest annotation in the dataset, this is the max lenght.

In [ ]:
from src.dataset import TDMSDataset, PATH


# load all annotations:
dataset = TDMSDataset(PATH.TRAIN)

annotations = []
for i in range(len(dataset)):
    f, tex, jsn = dataset[i]
    annotations.append(str(jsn))


In [ ]:
import pandas as pd


df = pd.DataFrame(annotations)
df.columns = ["annotation"]
df

In [ ]:
df["str_len"] = df.apply(lambda row: len(row["annotation"]), axis=1)

# top 10 longest annotations by string lenght:
df_t10 = df.sort_values(by="str_len", ascending=False)[:10]
df_t10

In [ ]:
# ignoring the outlier of 56561 chars and all below 20k chars as candidates for longest annotation

df_longest = df_t10[1:6]
df_longest


In [ ]:
# is the longest annotation character wise also the longest annotation token wise?
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

df_longest["token_len"] = df_longest.apply(lambda row: len(tokenizer(row["annotation"])["input_ids"]), axis=1)
df_longest["token_len"].max()

# longest = 9878 tokens => Max len should be 10k tokens

In [ ]:
# Sanity check: Other Tokenizers (llama)

from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

df_longest["token_len"] = df_longest.apply(lambda row: len(tokenizer(row["annotation"])["input_ids"]), axis=1)
df_longest["token_len"].max()

# longest = 8670 tokens => Max len should be 8700 tokens

In [ ]:
# Sanity check: Other Tokenizers (gemma)

from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b")

df_longest["token_len"] = df_longest.apply(lambda row: len(tokenizer(row["annotation"])["input_ids"]), axis=1)
df_longest["token_len"].max()

# longest = 8898 tokens => Max len should be 9k tokens

# Model Experiments

In [ ]:
# Llama 3 8b vs Mistral 7b vs Gemma 7b

In [2]:
from src.models import Model


mistral_it = Model("mistralai/Mistral-7B-Instruct-v0.3", 0)
gemma_it = Model("google/gemma-7b-it", 1)
# llama_it = HFModel("meta-llama/Meta-Llama-3-8B-Instruct", 2)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
mistral_it.generate("Give an example of a kind of fish, reply be stating the name only.")

In [3]:
# from src.content_extraction import naive_doctaet

# def extract_doctaet(model, prompt_template, tex):
#     doctaet = naive_doctaet(tex)
#     cut_doctaet = model.cut_text(doctaet, model.num_tokens(prompt_template("")))
#     prompt = prompt_template(cut_doctaet)
#     response = model.generate(prompt)
#     return response

In [8]:
simple_fs.__name__

'simple_fs'

In [4]:
from src.experiment_runner import Experiment, run
from src.dataset import PATH
from src.prompt_templates import simple_zs, simple_fs
from src.content_extraction import naive_doctaet


exps = [
    Experiment(mistral_it, simple_zs, naive_doctaet),
    Experiment(mistral_it, simple_fs, naive_doctaet),
    Experiment(gemma_it, simple_zs, naive_doctaet),
    Experiment(gemma_it, simple_fs, naive_doctaet),
]

dfs = []
for exp in exps:
    df = run(exp, PATH.VAL, max_iter=10)
    dfs.append(df)

# for exp in exps:
#     df = run(exp, PATH.TEST)
#     dfs.append(df)

  0%|                                                     | 0/10 [06:16<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
run = "VAL_mistral-it_fs_initial-20240610-221117"

import pandas as pd


df = pd.read_feather(f"results/{run}/df.feather")

for annotation in df["annotation"][:10]:
    print(annotation)